In [1]:
import os
BASE = "/content/drive/MyDrive/meeting-task-assigner"
os.makedirs(BASE, exist_ok=True)
print("Project folder:", BASE)


Project folder: /content/drive/MyDrive/meeting-task-assigner


In [2]:
!python --version
!df -h | sed -n '1,6p'


Python 3.12.12
Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   39G   70G  36% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
/dev/sda1        73G   40G   34G  54% /kaggle/input


In [1]:
# Colab shell cell
!apt-get update -y && apt-get install -y ffmpeg
!pip install --upgrade pip
!pip install openai-whisper pydub spacy dateparser fuzzywuzzy python-Levenshtein pytest tqdm
!python -m spacy download en_core_web_sm


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,491 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/p

In [6]:
import os, textwrap
BASE = "/content/drive/MyDrive/meeting-task-assigner"
folders = ["src","sample_data","tests","outputs"]
for f in folders:
    os.makedirs(os.path.join(BASE,f), exist_ok=True)

# Create placeholder files
files = {
    "README.md": "# Meeting Task Assigner\n\nProject workspace in Google Drive.",
    "requirements.txt": "openai-whisper\npydub\nspacy\ndateparser\nfuzzywuzzy\npython-Levenshtein\npytest\ntqdm\n",
    "src/transcribe.py": "# placeholder",
    "src/nlp_pipeline.py": "# placeholder",
    "src/assigner.py": "# placeholder",
    "sample_data/team.json": """[
  {"name":"Sakshi", "role": "Frontend Developer", "skills":["React","JavaScript","UI","frontend","ui bugs"]},
  {"name":"Mohit", "role": "Backend Engineer", "skills":["Database","APIs","Performance","backend","optimization"]},
  {"name":"Arjun", "role": "UI/UX Designer", "skills":["Figma","User flows","Mobile design","UI","UX"]},
  {"name":"Lata", "role": "QA Engineer", "skills":["Testing","Automation","Quality assurance","unit tests","QA"]}
]"""
}
for path, content in files.items():
    p = os.path.join(BASE, path)
    d = os.path.dirname(p)
    os.makedirs(d, exist_ok=True)
    with open(p, "w", encoding="utf-8") as fh:
        fh.write(content)
print("Project skeleton created at", BASE)


Project skeleton created at /content/drive/MyDrive/meeting-task-assigner


In [7]:
import importlib, sys, os
modules = ["whisper","pydub","spacy","dateparser","fuzzywuzzy","Levenshtein"]
for m in modules:
    try:
        importlib.import_module(m)
        print(m, "OK")
    except Exception as e:
        print(m, "FAILED:", e)

# show created files
!ls -la "/content/drive/MyDrive/meeting-task-assigner"
!ls -la "/content/drive/MyDrive/meeting-task-assigner/src"
!ls -la "/content/drive/MyDrive/meeting-task-assigner/sample_data"


whisper OK
pydub OK


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


spacy OK
dateparser OK
fuzzywuzzy OK
Levenshtein OK
total 17
drwx------ 2 root root 4096 Nov 29 03:39 outputs
-rw------- 1 root root   59 Nov 29 03:39 README.md
-rw------- 1 root root   80 Nov 29 03:39 requirements.txt
drwx------ 2 root root 4096 Nov 29 03:39 sample_data
drwx------ 2 root root 4096 Nov 29 03:39 src
drwx------ 2 root root 4096 Nov 29 03:39 tests
total 2
-rw------- 1 root root 13 Nov 29 03:39 assigner.py
-rw------- 1 root root 13 Nov 29 03:39 nlp_pipeline.py
-rw------- 1 root root 13 Nov 29 03:39 transcribe.py
total 1
-rw------- 1 root root 449 Nov 29 03:39 team.json


In [8]:
# create a synthetic meeting audio using gTTS (Google Text-to-Speech)
!pip install gTTS --quiet

from gtts import gTTS
import os, textwrap

meeting_text = textwrap.dedent("""
Hi everyone, let's discuss this week's priorities. Sakshi, we need someone to fix the critical login bug that users reported yesterday. This needs to be done by tomorrow evening since it's blocking users. Also, the database performance is really slow, Mohit you're good with backend optimization right? We should tackle this by end of this week, it's affecting the user experience. And we need to update the API documentation before Friday's release - this is high priority. Oh, and someone should design the new onboarding screens for the next sprint. Arjun, didn't you work on UI designs last month? This can wait until next Monday. One more thing - we need to write unit tests for the payment module. This depends on the login bug fix being completed first, so let's plan this for Wednesday.
""")

outdir = "/content/drive/MyDrive/meeting-task-assigner/sample_data"
os.makedirs(outdir, exist_ok=True)
outfile = os.path.join(outdir, "sample_meeting.mp3")
tts = gTTS(meeting_text)
tts.save(outfile)
print("Saved synthetic meeting audio to:", outfile)


Saved synthetic meeting audio to: /content/drive/MyDrive/meeting-task-assigner/sample_data/sample_meeting.mp3


In [9]:
import os
import whisper
from pathlib import Path
from datetime import datetime

DRIVE_BASE = "/content/drive/MyDrive/meeting-task-assigner"
SAMPLE_DIR = os.path.join(DRIVE_BASE, "sample_data")
OUT_DIR = os.path.join(DRIVE_BASE, "outputs")
os.makedirs(OUT_DIR, exist_ok=True)

# Choose the file to transcribe:
# If you uploaded a file via Option A, replace filename below with that file's name
# If you used Option B, the generated file is "sample_meeting.mp3"
filename = None
# auto-detect any audio file in sample_data if filename not manually set
if not filename:
    candidates = [p for p in os.listdir(SAMPLE_DIR) if p.lower().endswith(('.mp3','.wav','.m4a'))]
    if not candidates:
        raise FileNotFoundError("No audio files found in sample_data. Upload one or run the generator.")
    filename = candidates[0]
audio_path = os.path.join(SAMPLE_DIR, filename)
print("Transcribing:", audio_path)

# Load model (choose 'small' for speed; change to 'base'/'medium' if you prefer)
model = whisper.load_model("small")   # change to "base" or "medium" if you want higher accuracy

# If you encounter CUDA/FP16 errors, set fp16=False in transcribe
result = model.transcribe(audio_path, fp16=False)
transcript = result.get("text","").strip()

# Save transcript with timestamp
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
outpath = os.path.join(OUT_DIR, f"transcript_{ts}.txt")
with open(outpath, "w", encoding="utf-8") as f:
    f.write(transcript)
# Also create a stable file transcript.txt for pipeline use
stable = os.path.join(OUT_DIR, "transcript.txt")
with open(stable, "w", encoding="utf-8") as f:
    f.write(transcript)

print("Transcript saved to:", outpath)
print("Stable transcript path:", stable)
print("\n--- Transcript preview (first 800 chars) ---\n")
print(transcript[:800])


Transcribing: /content/drive/MyDrive/meeting-task-assigner/sample_data/sample_meeting.mp3


100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 50.3MiB/s]


Transcript saved to: /content/drive/MyDrive/meeting-task-assigner/outputs/transcript_20251129_040032.txt
Stable transcript path: /content/drive/MyDrive/meeting-task-assigner/outputs/transcript.txt

--- Transcript preview (first 800 chars) ---

Hi everyone, let's discuss this week's priorities. Sockshee, we need someone to fix the critical login bug that users reported yesterday. This needs to be done by tomorrow evening since it's blocking users. Also, the database performance is really slow. Mo hit your good with backend optimization, right? We should tackle this by end of this week. It's affecting the user experience. And we need to update the API documentation before Friday's release. This is high priority. Oh, and someone should design the new onboarding screens for the next sprint. Arjun, didn't you work on UI designs last month? This can wait until next Monday. One more thing, we need to write unit tests for the payment module. This depends on the login bug fix being completed fi

In [10]:
# Colab Step 3: NLP extraction + assignment
import os, json, re
from pathlib import Path
from datetime import datetime, timedelta
import dateparser
import spacy
from fuzzywuzzy import process

# --- Config / paths ---
BASE = "/content/drive/MyDrive/meeting-task-assigner"
TRANSCRIPT_PATH = os.path.join(BASE, "outputs", "transcript.txt")
TEAM_PATH = os.path.join(BASE, "sample_data", "team.json")
OUT_TASKS = os.path.join(BASE, "outputs", "tasks.json")

# Load spaCy
nlp = spacy.load("en_core_web_sm")

# Load transcript
with open(TRANSCRIPT_PATH, encoding="utf-8") as f:
    transcript = f.read().strip()

print("Transcript (first 400 chars):\n", transcript[:400], "\n---\n")

# Load team
with open(TEAM_PATH, encoding="utf-8") as f:
    team = json.load(f)

team_names = [m["name"] for m in team]

# Helper functions
PRIORITY_KEYWORDS = {
    "Critical": ["critical", "blocking", "urgent", "asap"],
    "High": ["high priority", "high priority.", "high", "important", "before friday", "before friday's release"],
    "Medium": ["medium", "next week", "next monday", "next monday.", "next sprint", "can wait", "plan", "by end of this week", "end of this week"],
    "Low": ["low", "whenever", "nice to have"]
}

TASK_VERB_KEYWORDS = ["fix", "update", "design", "write", "optimize", "test", "implement", "create", "tackle", "plan"]

def detect_priority(text):
    t = text.lower()
    for p, kws in PRIORITY_KEYWORDS.items():
        for kw in kws:
            if kw in t:
                return p
    # heuristic: very soon deadlines -> bump priority
    if any(x in t for x in ["tomorrow","by tomorrow","today","tonight"]):
        return "Critical"
    return "Medium"

def extract_deadline(text, ref_date=None):
    if ref_date is None:
        ref_date = datetime.now()
    # Use dateparser.search.search_dates to find phrases
    try:
        results = dateparser.search.search_dates(text, settings={'RELATIVE_BASE': ref_date, 'PREFER_DATES_FROM': 'future'})
    except Exception:
        results = None
    if results:
        # prefer the first relevant-looking date (some phrases return time ranges)
        phrase, dt = results[0]
        # Convert to ISO if possible
        try:
            iso = dt.isoformat()
            return iso, phrase
        except Exception:
            return str(dt), phrase
    # heuristic catches
    low = text.lower()
    if "end of this week" in low or "by end of this week" in low:
        # approximate as coming sunday of the same week
        base = ref_date
        days_to_sun = (6 - base.weekday()) % 7
        dt = (base + timedelta(days=days_to_sun)).replace(hour=23, minute=59, second=0, microsecond=0)
        return dt.isoformat(), "end of this week"
    return None, None

def extract_task_candidates(transcript):
    doc = nlp(transcript)
    sentences = [sent.text.strip() for sent in doc.sents]
    candidates = []
    for s in sentences:
        s_lower = s.lower()
        # heuristics: look for task-like phrases
        if re.search(r"\b(we need|someone should|please|let's|we should|should|we need to|we need someone|we need someone to)\b", s_lower):
            candidates.append(s)
            continue
        if any(kw in s_lower for kw in TASK_VERB_KEYWORDS):
            candidates.append(s)
            continue
    # dedupe while preserving order
    seen = set()
    out = []
    for s in candidates:
        if s not in seen:
            out.append(s)
            seen.add(s)
    return out

def assign_assignee(sentence, team_members, name_threshold=70):
    # 1) explicit name fuzzy match
    names = [m['name'] for m in team_members]
    best = process.extractOne(sentence, names)
    if best and best[1] >= name_threshold:
        name = best[0]
        return name, f"explicit_mention (score={best[1]})"
    # 2) skill match: check if any skill keyword appears
    s = sentence.lower()
    for m in team_members:
        for skill in m.get("skills", []):
            if skill.lower() in s:
                return m["name"], f"skill_match:{skill}"
    # 3) keyword-to-role heuristics
    if any(k in s for k in ["ui", "onboarding", "design", "figma", "ux", "ui designs", "mobile design", "screens"]):
        # prefer UI/UX designer
        for m in team_members:
            if "designer" in m.get("role","").lower() or "ui" in m.get("role","").lower():
                return m["name"], "heuristic_role_match:ui/ux"
    if any(k in s for k in ["database","db","backend","api","performance","optimiz"]):
        for m in team_members:
            if "backend" in m.get("role","").lower() or "engineer" in m.get("role","").lower():
                return m["name"], "heuristic_role_match:backend"
    if any(k in s for k in ["test","unit test","testing","qa","automation","payment module"]):
        for m in team_members:
            if "qa" in m.get("role","").lower() or "quality" in m.get("role","").lower():
                return m["name"], "heuristic_role_match:qa"
    return None, "no_match"

def title_from_sentence(s):
    # create a concise title by trimming to verb+object using a regex heuristic
    s = s.strip()
    # remove leading vocatives like "Sakshi," or "Arjun,"
    s = re.sub(r'^[A-Z][a-z]+,\s*', '', s)
    # attempt to capture verb phrase
    m = re.search(r"(fix|update|design|write|optimize|test|implement|create|tackle|plan)\s+(.+)", s, flags=re.IGNORECASE)
    if m:
        verb = m.group(1).capitalize()
        obj = m.group(2)
        title = f"{verb} {obj}"
        title = title[:100]
        return title
    # fallback: first 60 chars
    return s[:80]

# --- Extract candidates ---
candidates = extract_task_candidates(transcript)
print(f"Found {len(candidates)} candidate sentences for tasks.\n")
for i,s in enumerate(candidates,1):
    print(i, "-", s)

# --- Build tasks ---
meeting_date = datetime.now()  # use current notebook time as reference
tasks = []
id_counter = 1
for sent in candidates:
    title = title_from_sentence(sent)
    priority = detect_priority(sent)
    deadline_iso, deadline_phrase = extract_deadline(sent, ref_date=meeting_date)
    assignee, reason_tag = assign_assignee(sent, team, name_threshold=65)  # slightly lower threshold for ASR errors
    task = {
        "id": id_counter,
        "title": title,
        "description": sent,
        "assigned_to": assignee,
        "deadline": deadline_iso if deadline_iso else (deadline_phrase if deadline_phrase else None),
        "priority": priority,
        "dependencies": [],
        "reasoning": reason_tag
    }
    tasks.append(task)
    id_counter += 1

# --- Dependency detection (simple) ---
# Link tasks that mention "depends on" or reference other tasks by keywords
for t in tasks:
    desc = t["description"].lower()
    if "depends on" in desc or "depend" in desc or "after" in desc:
        # attempt to find referenced task by searching for keywords like 'login' or 'payment'
        for other in tasks:
            if other["id"] == t["id"]:
                continue
            # keyword match between descriptions
            for token in ["login","payment","database","api","onboarding","unit test","tests","bug"]:
                if token in desc and token in other["description"].lower():
                    t["dependencies"].append(other["id"])
                    break

# --- Post-process: if assignee None, try skill-match across team more aggressively ---
for t in tasks:
    if not t["assigned_to"]:
        # look for top skill match
        s = t["description"].lower()
        best_match = None
        for m in team:
            score = 0
            for skill in m.get("skills", []):
                if skill.lower() in s:
                    score += 1
            # also check role keywords
            if m.get("role"):
                if m["role"].lower().split()[0] in s:
                    score += 0.5
            if score > 0 and (best_match is None or score > best_match[1]):
                best_match = (m["name"], score)
        if best_match:
            t["assigned_to"] = best_match[0]
            t["reasoning"] = f"skill_fallback ({best_match[1]:.1f})"

# --- Save tasks to disk ---
output = {
    "meeting_file": os.path.basename(TRANSCRIPT_PATH),
    "extracted_at": meeting_date.isoformat(),
    "transcript_preview": transcript[:800],
    "tasks": tasks
}
with open(OUT_TASKS, "w", encoding="utf-8") as fh:
    json.dump(output, fh, indent=2)

print("\nSaved tasks to:", OUT_TASKS)

# --- Pretty print tasks table ---
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["id","title","assigned_to","deadline","priority","dependencies","reasoning"]
for t in tasks:
    table.add_row([t["id"], t["title"][:30], t["assigned_to"], t["deadline"], t["priority"], t["dependencies"], t["reasoning"]])
print("\nExtracted tasks:\n")
print(table)

# Also print JSON summary
print("\nJSON tasks summary (first 400 chars):\n")
print(json.dumps(output, indent=2)[:400])


Transcript (first 400 chars):
 Hi everyone, let's discuss this week's priorities. Sockshee, we need someone to fix the critical login bug that users reported yesterday. This needs to be done by tomorrow evening since it's blocking users. Also, the database performance is really slow. Mo hit your good with backend optimization, right? We should tackle this by end of this week. It's affecting the user experience. And we need to u 
---

Found 9 candidate sentences for tasks.

1 - Hi everyone, let's discuss this week's priorities.
2 - Sockshee, we need someone to fix the critical login bug that users reported yesterday.
3 - We should tackle this by end of this week.
4 - And we need to update the API documentation before Friday's release.
5 - Oh, and someone should design the new onboarding screens for the next sprint.
6 - Arjun, didn't you work on UI designs last month?
7 - One more thing, we need to write unit tests for the payment module.
8 - This depends on the login bug fix being compl

In [13]:
# Improved extraction + assignment (run in Colab)
import os, json, re
from pathlib import Path
from datetime import datetime, timedelta
import dateparser
import spacy
from fuzzywuzzy import process, fuzz
from collections import defaultdict

BASE = "/content/drive/MyDrive/meeting-task-assigner"
TRANSCRIPT_PATH = os.path.join(BASE, "outputs", "transcript.txt")
TEAM_PATH = os.path.join(BASE, "sample_data", "team.json")
OUT_TASKS = os.path.join(BASE, "outputs", "tasks_refined.json")

nlp = spacy.load("en_core_web_sm")

with open(TRANSCRIPT_PATH, encoding="utf-8") as f:
    transcript = f.read().strip()

with open(TEAM_PATH, encoding="utf-8") as f:
    team = json.load(f)
team_names = [m["name"] for m in team]

# Useful helpers
PRIORITY_KEYWORDS = {
    "Critical": ["critical", "blocking", "urgent", "asap"],
    "High": ["high priority", "high", "important", "before friday", "before friday's release"],
    "Medium": ["medium", "next week", "next monday", "next sprint", "can wait", "plan", "end of this week"],
    "Low": ["low", "whenever", "nice to have"]
}
TASK_VERB_KEYWORDS = ["fix","update","design","write","optimize","test","implement","create","tackle","plan","add","remove","investigate","refactor"]

def detect_priority(text):
    t = text.lower()
    for p,kws in PRIORITY_KEYWORDS.items():
        for kw in kws:
            if kw in t:
                return p
    if any(x in t for x in ["tomorrow","by tomorrow","today","tonight"]):
        return "Critical"
    return "Medium"

def extract_date_from_text(text, ref_date=None):
    if ref_date is None:
        ref_date = datetime.now()
    try:
        results = dateparser.search.search_dates(text, settings={'RELATIVE_BASE': ref_date, 'PREFER_DATES_FROM':'future'})
    except Exception:
        results = None
    if results:
        phrase, dt = results[0]
        return dt.isoformat(), phrase
    # heuristics
    low = text.lower()
    if "end of this week" in low or "by end of this week" in low:
        base = ref_date
        days_to_sun = (6 - base.weekday()) % 7
        dt = (base + timedelta(days=days_to_sun)).replace(hour=23, minute=59, second=0, microsecond=0)
        return dt.isoformat(), "end of this week"
    return None, None

# --- REPLACE existing find_vocative_name with this improved function ---
BAD_VOCATIVES = set(["and","but","oh","so","also","well","hey","hi","hello","one","alright"])

def find_vocative_name(sentence):
    # match "Name," at start but ignore short/common words
    m = re.match(r'^\s*([A-Z][a-zA-Z]{2,})(?:\s+|,)', sentence)
    if m:
        tok = m.group(1)
        if tok.lower() not in BAD_VOCATIVES and len(tok) > 2:
            return tok
    # match patterns like "Sakshi," or "Arjun," later in sentence beginning
    m2 = re.match(r'^\s*([A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{1,})?),', sentence)
    if m2:
        tok = m2.group(1).replace(" ", "")
        if tok.lower() not in BAD_VOCATIVES and len(tok) > 2:
            return tok
    return None


def normalize_name_token(tok):
    # remove non-letters, join split pieces like "Mo hit" -> "Mohit"
    s = re.sub(r'[^A-Za-z]', '', tok)
    return s

def best_team_name_match(token, names, threshold=65):
    if not token:
        return None, 0
    token_clean = normalize_name_token(token)
    # Try direct fuzzy
    best = process.extractOne(token_clean, names, scorer=fuzz.token_sort_ratio)
    if best and best[1] >= threshold:
        return best[0], best[1]
    # try join-split variants (e.g., "Mo hit" -> "Mohit")
    joined = token.replace(" ", "")
    best2 = process.extractOne(joined, names, scorer=fuzz.token_sort_ratio)
    if best2 and best2[1] >= threshold:
        return best2[0], best2[1]
    # lower threshold fallback if token is short but close
    if best and best[1] >= (threshold - 10):
        return best[0], best[1]
    return None, 0

def extract_task_sentences(transcript):
    doc = nlp(transcript)
    sents = [sent.text.strip() for sent in doc.sents]
    candidates_idx = []
    for i,s in enumerate(sents):
        sl = s.lower()
        if re.search(r"\b(we need|someone should|please|let's|we should|should|we need to|we need someone|we need someone to|one more thing)\b", sl):
            candidates_idx.append(i)
            continue
        if any(kw in sl for kw in TASK_VERB_KEYWORDS):
            candidates_idx.append(i)
            continue
    # dedupe preserving order
    seen=set(); out=[]
    for i in candidates_idx:
        if i not in seen:
            out.append(i); seen.add(i)
    return sents, out

# Merge cluster: attach short referential sentences and deadline-bearing neighbors
def build_clusters(sents, candidate_idxs):
    clusters = []  # list of dicts {idxs:list, text: str}
    i = 0
    for idx in candidate_idxs:
        # start cluster at idx, then attach following sentences if they are referential/short or contain date words
        cluster_idxs = [idx]
        # look ahead up to 2 sentences
        for j in range(1,3):
            if idx + j >= len(sents):
                break
            next_s = sents[idx+j].strip()
            low = next_s.lower()
            # attach if it's referential or contains deadlines/dependency words
            if len(next_s.split()) < 6 or any(w in low for w in ["tomorrow","wednesday","friday","monday","by","depends on","depends","depend","so let's","plan","this needs","this is","it's","it's affecting","end of this week"]):
                cluster_idxs.append(idx+j)
            else:
                break
        cluster_text = " ".join([sents[k] for k in cluster_idxs])
        clusters.append({"idxs":cluster_idxs, "text": cluster_text, "first_idx": idx})
    return clusters

# Build candidates
sents, candidate_idxs = extract_task_sentences(transcript)
clusters = build_clusters(sents, candidate_idxs)

# For debug: show clusters
print(f"Found {len(clusters)} task clusters.\n")
for i,c in enumerate(clusters,1):
    print(i,"(sent idxs:",c['idxs'],") ->", c['text'])

# Assignment and extraction
tasks = []
idc = 1
meeting_date = datetime.now()
for c in clusters:
    text = c["text"]
    # detect vocative
    voc = find_vocative_name(sents[c["first_idx"]])
    assignee = None
    reasoning = None
    if voc:
        name, score = best_team_name_match(voc, team_names, threshold=60)
        if name:
            assignee = name
            reasoning = f"vocative_explicit ({voc}, score={score})"
    # if not assigned, try fuzzy match on full cluster text for names
    if not assignee:
        best = process.extractOne(text, team_names, scorer=fuzz.token_sort_ratio)
        if best and best[1] >= 70:
            assignee = best[0]
            reasoning = f"explicit_mention_full ({best[1]})"
    # fallback skill matching
    if not assignee:
        tl = text.lower()
        for m in team:
            for skill in m.get("skills",[]):
                if skill.lower() in tl:
                    assignee = m["name"]
                    reasoning = f"skill_match:{skill}"
                    break
            if assignee: break
    # heuristic role keywords
    if not assignee:
        tl = text.lower()
        if any(k in tl for k in ["ui","onboarding","design","figma","ux","screens"]):
            for m in team:
                if "designer" in m.get("role","").lower() or "ui" in m.get("role","").lower():
                    assignee = m["name"]; reasoning = "heuristic_role:ui/ux"; break
        if not assignee and any(k in tl for k in ["database","db","backend","api","performance","optimiz"]):
            for m in team:
                if "backend" in m.get("role","").lower() or "engineer" in m.get("role","").lower():
                    assignee = m["name"]; reasoning = "heuristic_role:backend"; break
        if not assignee and any(k in tl for k in ["test","unit test","testing","qa","automation","payment"]):
            for m in team:
                if "qa" in m.get("role","").lower() or "quality" in m.get("role","").lower():
                    assignee = m["name"]; reasoning = "heuristic_role:qa"; break

    # priority & deadline
    priority = detect_priority(text)
    deadline_iso, deadline_phrase = extract_date_from_text(text, ref_date=meeting_date)
    # build title
    # remove leading vocative tokens for title readability
    title_text = re.sub(r'^[A-Z][a-zA-Z]+\s*,\s*', '', text)
    m = re.search(r"(fix|update|design|write|optimi(?:z|s)e|test|implement|create|tackle|plan)\s+(.+?)(?:\.|$)", title_text, flags=re.IGNORECASE)
    if m:
        title = f"{m.group(1).capitalize()} {m.group(2)}"
        title = title[:120]
    else:
        title = title_text[:100]

    tasks.append({
        "id": idc,
        "title": title,
        "description": text,
        "assigned_to": assignee,
        "deadline": deadline_iso if deadline_iso else (deadline_phrase if deadline_phrase else None),
        "deadline_phrase": deadline_phrase,
        "priority": priority,
        "dependencies": [],
        "reasoning": reasoning or "no_match"
    })
    idc += 1

# Dependency linking by similarity of keywords
for t in tasks:
    desc = t["description"].lower()
    if any(k in desc for k in ["depend", "depends on", "after"]):
        for other in tasks:
            if other["id"] == t["id"]:
                continue
            # try to match by important tokens
            for token in ["login","payment","database","api","onboarding","unit test","tests","bug","login bug","payment module"]:
                if token in desc and token in other["description"].lower():
                    t["dependencies"].append(other["id"])
                    break

# Save refined output
output = {
    "meeting_file": os.path.basename(TRANSCRIPT_PATH),
    "extracted_at": meeting_date.isoformat(),
    "transcript_preview": transcript[:1000],
    "tasks": tasks
}
with open(OUT_TASKS, "w", encoding="utf-8") as fh:
    json.dump(output, fh, indent=2)

# pretty print
from prettytable import PrettyTable
t = PrettyTable()
t.field_names = ["id","title","assigned_to","deadline","priority","dependencies","reasoning"]
for x in tasks:
    t.add_row([x["id"], x["title"][:40], x["assigned_to"], x["deadline"], x["priority"], x["dependencies"], x["reasoning"]])
print("\nRefined extracted tasks:\n")
print(t)
print("\nSaved refined tasks to:", OUT_TASKS)


Found 9 task clusters.

1 (sent idxs: [0] ) -> Hi everyone, let's discuss this week's priorities.
2 (sent idxs: [1, 2] ) -> Sockshee, we need someone to fix the critical login bug that users reported yesterday. This needs to be done by tomorrow evening since it's blocking users.
3 (sent idxs: [5, 6, 7] ) -> We should tackle this by end of this week. It's affecting the user experience. And we need to update the API documentation before Friday's release.
4 (sent idxs: [7, 8] ) -> And we need to update the API documentation before Friday's release. This is high priority.
5 (sent idxs: [9] ) -> Oh, and someone should design the new onboarding screens for the next sprint.
6 (sent idxs: [10, 11] ) -> Arjun, didn't you work on UI designs last month? This can wait until next Monday.
7 (sent idxs: [12, 13, 14] ) -> One more thing, we need to write unit tests for the payment module. This depends on the login bug fix being completed first. So let's plan this for Wednesday.
8 (sent idxs: [13, 14] 

In [14]:
%%bash
# create test file
cat > /content/drive/MyDrive/meeting-task-assigner/tests/test_pipeline_basic.py <<'PY'
import json
from pathlib import Path

BASE = Path("/content/drive/MyDrive/meeting-task-assigner")
OUT = BASE / "outputs" / "tasks_refined.json"

def test_tasks_refined_exists():
    assert OUT.exists(), "tasks_refined.json missing; run pipeline first"

def test_expected_task_count():
    data = json.loads(OUT.read_text(encoding='utf-8'))
    tasks = data['tasks']
    # We'll expect at least 6 useful tasks from this meeting
    assert len(tasks) >= 6, f"Expected at least 6 tasks, found {len(tasks)}"

def test_core_assignments_present():
    data = json.loads(OUT.read_text(encoding='utf-8'))
    names = set(t.get('assigned_to') for t in data['tasks'] if t.get('assigned_to'))
    # Ensure key team members appear as assignees somewhere
    assert "Sakshi" in names, "Sakshi should be assigned (login bug)"
    assert "Mohit" in names, "Mohit should be assigned (DB/API)"
    assert "Lata" in names, "Lata should be assigned (unit tests)"
    assert "Arjun" in names, "Arjun should be assigned (UI tasks)"
PY

# run pytest (quiet)
pytest -q /content/drive/MyDrive/meeting-task-assigner/tests/test_pipeline_basic.py -q || true


...                                                                      [100%]
